In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
import tqdm

from james.msmarco.ms_marco_eval import load_reference, compute_metrics_james_zhao

In [3]:
residing_folder = "./data/results/baseline"
knn_index_file = "knn.npy"
output_file = "inference.tsv"

In [8]:
inference_dev_ids = pd.read_csv("./data/qrels/qrels_with_text.tsv", header=None, sep="\t", names=["qid", "query"])
knn_results = np.load(os.path.join(residing_folder, knn_index_file))
print(f"{len(inference_dev_ids)=}, {len(knn_results)=}")
inference_dev_ids.head()

len(inference_dev_ids)=55578, len(knn_results)=55578


,qid,query
0,1102432,. what is a corporation?
1,1102431,why did rachel carson write an obligation to e...
2,1090282,symptoms of a dying mouse
3,39449,average number of lightning strikes per day
4,76162,can you burn your lawn with fertilizer


In [10]:
# qid to ranked ids
inference = {}
for qid, knn_row in zip(inference_dev_ids["qid"], knn_results):
  inference[qid]= knn_row.tolist()
reference = load_reference("./data/qrels/qrels.dev.tsv")
assert(set(inference.keys())==set(reference.keys()))

In [12]:
scores, ranking = compute_metrics_james_zhao(
  qids_to_relevant_passageids=reference,
  qids_to_ranked_candidate_passages=inference,
  max_mrr_rank=1000,
)

In [14]:
print(scores)

{'MRR @1000': 0.30519164437672375, 'QueriesRanked': 55578}
